<a href="https://colab.research.google.com/github/Aryan-raj05/Alarm_prediction_alert/blob/main/Ml_ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving AlarmList_HistoricalAlarms_1_DEC_2024_1.xlsx to AlarmList_HistoricalAlarms_1_DEC_2024_1.xlsx
Saving AlarmList_HistoricalAlarms_1_DEC_2024_2.xlsx to AlarmList_HistoricalAlarms_1_DEC_2024_2.xlsx
Saving AlarmList_HistoricalAlarms_2_DEC_2024_1.xlsx to AlarmList_HistoricalAlarms_2_DEC_2024_1.xlsx
Saving AlarmList_HistoricalAlarms_2_DEC_2024_2.xlsx to AlarmList_HistoricalAlarms_2_DEC_2024_2.xlsx
Saving AlarmList_HistoricalAlarms_4_DEC_2024_1.xlsx to AlarmList_HistoricalAlarms_4_DEC_2024_1.xlsx
Saving AlarmList_HistoricalAlarms_5_DEC_2024_1.xlsx to AlarmList_HistoricalAlarms_5_DEC_2024_1.xlsx
Saving AlarmList_HistoricalAlarms_5_DEC_2024_2.xlsx to AlarmList_HistoricalAlarms_5_DEC_2024_2.xlsx
Saving AlarmList_HistoricalAlarms_6_DEC_2024_1.xlsx to AlarmList_HistoricalAlarms_6_DEC_2024_1.xlsx
Saving AlarmList_HistoricalAlarms_7_DEC_2024_1.xlsx to AlarmList_HistoricalAlarms_7_DEC_2024_1.xlsx


In [14]:
import pandas as pd
df=pd.read_csv("/content/merged_dataset.csv")

In [20]:
df.columns

Index(['Unnamed: 0', 'Severity', 'Site Name', 'Source System',
       'Life Span (minutes)', 'Alarm Name', 'First Time Detected',
       'Last Time Cleared', 'Alarmed Object Name', 'Last Time Detected',
       'Alarmed Object Type', 'Alarmed Object Source System', 'Alarm Type',
       'Probable Cause', 'Specific Problem', 'Is Service Affecting',
       'Alarm ID', 'Previous Severity', 'Number Of Occurrences',
       'Additional Text'],
      dtype='object')

In [21]:
# Drop rows missing critical categorical identifiers
df = df.dropna(subset=["Site Name", "Alarm Name"])

# Handle Specific Problem
df["Specific Problem"] = df["Specific Problem"].fillna("UNKNOWN")
df["specific_problem_missing"] = df["Specific Problem"] == "UNKNOWN"

# Drop Additional Text completely
df = df.drop(columns=["Additional Text"], errors="ignore")

# Handle Previous Severity
if "Previous Severity" in df.columns:
    df = df.dropna(subset=["Previous Severity"])  # or fill with mode

# Handle Number of Occurrences
if "Number Of Occurrence" in df.columns:
    df["Number Of Occurrence"] = df["Number Of Occurrence"].fillna(0).astype(int)


<ipython-input-21-8366af20ccce>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Specific Problem"] = df["Specific Problem"].fillna("UNKNOWN")
<ipython-input-21-8366af20ccce>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["specific_problem_missing"] = df["Specific Problem"] == "UNKNOWN"


In [25]:
print(df_unique.columns.tolist())
# print(df_unique.head(3))


['Unnamed: 0', 'Severity', 'Site Name', 'Source System', 'Life Span (minutes)', 'Alarm Name', 'First Time Detected', 'Last Time Cleared', 'Alarmed Object Name', 'Last Time Detected', 'Alarmed Object Type', 'Alarmed Object Source System', 'Alarm Type', 'Probable Cause', 'Specific Problem', 'Is Service Affecting', 'Alarm ID', 'Previous Severity', 'Number Of Occurrences', 'Additional Text']


In [22]:
unique_values_dict = {}

# Iterate through each column and extract unique values
for column in df.columns:
    unique_values = df[column].unique().tolist()  # Get unique values as a list
    unique_values_dict[column] = unique_values    # Store in dictionary

# Convert the dictionary to a DataFrame (for easy CSV export)
unique_values_df = pd.DataFrame.from_dict(unique_values_dict, orient='index').transpose()

# Save to CSV
unique_values_df.to_csv('unique_values_per_column.csv', index=False)
from google.colab import files
files.download("/content/unique_values_per_column.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
import pandas as pd

# Load original full dataset (not the unique-values one)
df = pd.read_csv("/content/merged_dataset.csv")  # Replace with your path

# Clean column names
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(r"\s+", "_", regex=True)
)

# Create ML-ready column list based on unique value distribution
ml_ready_cols = []
for col in df.columns:
    n_unique = df[col].nunique(dropna=False)
    if n_unique == 1:
        continue  # skip constant columns
    elif pd.api.types.is_numeric_dtype(df[col]):
        ml_ready_cols.append(col)  # numerical features are ready
    elif n_unique == 2:
        ml_ready_cols.append(col)  # binary categorical
    elif n_unique <= 100:
        ml_ready_cols.append(col)  # one-hot or label-encoded features
    else:
        pass  # too high-cardinality, maybe process later via NLP/embedding(>100 unique valuse)

# Filter dataset with ML-usable columns
df_ml_ready = df[ml_ready_cols].copy()

# Show the list of selected columns
print(" Columns selected for ML training:")
print(df_ml_ready.columns.tolist())

# Show shape
print(f" ML-ready dataset shape: {df_ml_ready.shape}")


✅ Columns selected for ML training:
['unnamed:_0', 'severity', 'source_system', 'life_span_(minutes)', 'alarmed_object_type', 'alarm_type', 'is_service_affecting', 'previous_severity', 'number_of_occurrences']
✅ ML-ready dataset shape: (2199207, 9)


## This Part is for me yours is just above part

In [ ]:
# --------------------------------------------
# alarm_preprocessing_merged.py
# --------------------------------------------
"""
Pre-process a single, merged Nokia alarm dataset (CSV) for ML modeling.

Outputs:
1. alarms_clean.parquet       – Cleaned, feature-engineered dataset
2. alarms_ml_ready.pkl        – (X, y_dict, preprocessor) for model training
"""

import os
import joblib
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer


def load_merged_data(path: str) -> pd.DataFrame:
    if path.endswith(".csv"):
        return pd.read_csv(path)


def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardize column names: remove spaces, lowercase, replace with underscores.
    """
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(r"\s+", "_", regex=True)
    )
    return df


def convert_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert timestamp and numeric fields to proper dtypes.
    """
    time_cols = ["first_time_detected", "last_time_detected", "last_time_cleared"]
    num_cols = ["number_of_occurrences", "life_span_(minutes)"]

    for col in time_cols:
        if col in df:
            df[col] = pd.to_datetime(df[col], errors="coerce")

    for col in num_cols:
        if col in df:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    return df


def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create new features based on time and alarm activity.
    """
    # 1. Is the alarm still active?
    df["is_active"] = df["last_time_cleared"].isna().astype(int)

    # 2. Decompose first_time_detected into time components
    dt = df["first_time_detected"]
    df["ftd_hour"] = dt.dt.hour
    df["ftd_weekday"] = dt.dt.dayofweek
    df["ftd_month"] = dt.dt.month
    df["ftd_year"] = dt.dt.year

    # 3. Inter-arrival time in minutes (per site)
    df = df.sort_values(["site_name", "first_time_detected"])
    df["interarrival_mins"] = (
        df.groupby("site_name")["first_time_detected"]
          .diff().dt.total_seconds().div(60)
    )

    return df


def drop_sparse_columns(df: pd.DataFrame) -> pd.DataFrame:
    domain_keep = {
        "last_time_cleared",
        "first_time_detected",
        "last_time_detected",
        "site_name",
        "alarm_type",
        "severity",
        "interarrival_mins"
    }
    missing_ratio = df.isna().mean()
    to_drop = [
        col for col, pct in missing_ratio.items()
        if pct > 0.95 and col not in domain_keep
    ]

    if to_drop:
        print("Dropping sparse columns:", to_drop)

    return df.drop(columns=to_drop)



def define_targets(df: pd.DataFrame) -> dict:
    """
    Define supervised learning targets based on current and next alarm state.
    """
    return {
        "next_alarm_type"      : df["alarm_type"].shift(-1),
        "future_alarm_occurs"  : (df["interarrival_mins"] < 60)
                                    .fillna(0).astype(int),
        "time_to_next_alarm"   : df["interarrival_mins"],
        "future_alarm_severity": df["severity"].shift(-1),
        "escalation_risk"      : (df["severity"] == "Critical").astype(int)
    }




def build_preprocessing_pipeline(df: pd.DataFrame):
    categorical = [
        "site_name", "alarmed_object_name", "alarmed_object_type",
        "source_system", "alarm_name", "alarm_type", "severity",
        "probable_cause", "specific_problem", "is_service_affecting"
    ]
    numeric = [
        "number_of_occurrences", "life_span_(minutes)",
        "interarrival_mins", "ftd_hour", "ftd_weekday",
        "ftd_month", "ftd_year", "is_active"
    ]

    for col in categorical:
        missing_flag = f"{col}_missing"
        df[missing_flag] = df[col].isna().astype(int)
        numeric.append(missing_flag)

    num_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median"))
    ])

    cat_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
        ("to_str", FunctionTransformer(lambda x: x.astype(str))),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
    ])

    preprocessor = ColumnTransformer([
        ("num", num_pipe, numeric),
        ("cat", cat_pipe, categorical)
    ], remainder="drop")


    return preprocessor, numeric + categorical



# def preprocess_pipeline(input_path: str):
#     """
#     Full preprocessing workflow: clean, transform, engineer features, and save outputs.
#     """
#     print("Loading merged dataset...")
#     df = load_merged_data(input_path)

#     print("Standardizing column names and converting types...")
#     df = clean_column_names(df)
#     df = convert_dtypes(df)

#     print("Engineering domain-specific features...")
#     df = drop_sparse_columns(df)
#     print("Cleaned column names:", df.columns.tolist())
#     df = engineer_features(df)

#     print("Creating supervised learning targets...")
#     y_dict = define_targets(df)

#     print("Building and applying scikit-learn preprocessing pipeline...")
#     preprocessor, feature_cols = build_preprocessing_pipeline(df)
#     X = preprocessor.fit_transform(df)

#     print("Saving preprocessed artifacts...")
#     df.to_parquet("alarms_clean.parquet", index=False)
#     joblib.dump((X, y_dict, preprocessor), "alarms_ml_ready.pkl")

#     print("Done. Outputs saved as:")
#     print(" - alarms_clean.parquet")
#     print(" - alarms_ml_ready.pkl")


# if __name__ == "__main__":
#     merged_path = "/home/sohanx1/Downloads/6th sem/SE/nokia/codes/gshare/merged_dataset.csv"  # Update path if needed
# preprocess_pipeline(merged_path)


In [ ]:

"""
Full preprocessing workflow: clean, transform, engineer features, and save outputs.
"""
print("Loading merged dataset...")
df = load_merged_data("/content/merged_dataset.csv")



Loading merged dataset...


In [ ]:
df = clean_column_names(df)
df = convert_dtypes(df)
df = drop_sparse_columns(df)

<ipython-input-2-7161f569f377>:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
<ipython-input-2-7161f569f377>:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
<ipython-input-2-7161f569f377>:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


Dropping sparse columns: ['alarmed_object_source_system']


In [ ]:
df = engineer_features(df)

In [ ]:
y_dict = define_targets(df)
print("Building and applying scikit-learn preprocessing pipeline...")
preprocessor, feature_cols = build_preprocessing_pipeline(df)
X = preprocessor.fit_transform(df)

Building and applying scikit-learn preprocessing pipeline...


/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['interarrival_mins' 'ftd_hour' 'ftd_weekday' 'ftd_month' 'ftd_year']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [ ]:
df.to_csv("cleaned_data.csv",index=False)

AttributeError: 'DataFrame' object has no attribute 'download'

In [ ]:
from google.colab import files

files.download("/content/cleaned_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>